## Cocomo Nasa V1

In [ ]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff

#algoritmos ia
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm 
from sklearn.tree import DecisionTreeRegressor

from sklearn import model_selection as ms
from sklearn.model_selection import GridSearchCV

## Preparando Dados

In [ ]:
#Carregando dados

data, meta = arff.loadarff('datasets/cocomonasa_v1.arff')
df = pd.DataFrame(data)
df.head()

In [ ]:
#removendo linhas com atributos nulos ou vazios

df = df.dropna()

#Convertendo atributos textuais em numeric com escala de atributos (ainda sem normalização) "OrdinalEncoder"

tranform_data = {b"Very_Low": 0, b"Low": 1, b"Nominal": 2, b"High": 3, b"Very_High": 4, b"Extra_High": 5}
df = df.replace(tranform_data)

#normalizando dados - #min-max df=(df-df.min())/(df.max()-df.min()) - #zscore df=(df-df.mean())/df.std()

df = (df-df.mean())/df.std()

#separando atributo classe ACT_Effort para entrada nos modelos #df.columns[-1]

X = df.drop(df.columns[-1], axis=1)
y = df.take([-1], axis=1).values.ravel()

## Treinando e avaliando o desempenho dos modelos

In [103]:
cv = ms.KFold(n_splits=3, shuffle=True, random_state=1)

models = []
#Instanciando os modelos #0 - nome, 1 - instância, 2 - parâmetros
models.append([' DT', DecisionTreeRegressor(), { "criterion" : ["squared_error", "friedman_mse", "absolute_error"],
                                                "splitter": ["best", "random"],
                                                "max_depth": np.arange(4,12),
                                                "random_state": np.arange(2,6)
                                                }])
models.append(['KNN', KNeighborsRegressor(), {  "n_neighbors" : np.arange(1,10),
                                                "weights": ["uniform", "distance"],
                                                "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']
                                                }])
models.append(['MLP', MLPRegressor(), {"hidden_layer_sizes": np.arange(10,100,10),
                                                "activation": ['identity', 'logistic', 'tanh', 'relu'],
                                                "solver": ['lbfgs', 'sgd', 'adam'],
                                                "random_state": [5], #  np.arange(2,6), fixado para reduzir tempo de processamento.
                                                "max_iter": [1500]
                                                }])
models.append(['RFR', RandomForestRegressor(), {"n_estimators": np.arange(5,20,5),
                                                    "max_depth" : np.arange(2,14,2),
                                                    "random_state": np.arange(2,6),
                                                    }])
models.append(['SVR', svm.SVR(), { "kernel" : [ 'poly', 'linear', 'rbf', 'sigmoid'], 
                                        "C": np.arange(2,14,2), 
                                        "epsilon": np.arange(0.001,0.1,0.001), # 0.1 a 0.9
                                    }])

In [104]:
for model in models:
    clf = GridSearchCV(model[1], model[2], scoring='neg_mean_absolute_error',  cv=cv,  n_jobs=-1)
    clf.fit(X, y)
    print(">", model[0], ">------ MAE: %.3f" % (1 + clf.best_score_), " ----- STD: %.3f" % (clf.cv_results_['std_test_score'][clf.best_index_]), " ----- Best Params:", (clf.best_params_))

>  DT >------ MAE: 0.798  ----- STD: 0.013  ----- Best Params: {'criterion': 'absolute_error', 'max_depth': 9, 'random_state': 3, 'splitter': 'random'}
> KNN >------ MAE: 0.608  ----- STD: 0.133  ----- Best Params: {'algorithm': 'auto', 'n_neighbors': 8, 'weights': 'distance'}
> MLP >------ MAE: 0.783  ----- STD: 0.055  ----- Best Params: {'activation': 'tanh', 'hidden_layer_sizes': 50, 'max_iter': 1500, 'random_state': 5, 'solver': 'lbfgs'}
> RFR >------ MAE: 0.808  ----- STD: 0.047  ----- Best Params: {'max_depth': 10, 'n_estimators': 5, 'random_state': 4}
> SVR >------ MAE: 0.757  ----- STD: 0.020  ----- Best Params: {'C': 12, 'epsilon': 0.004, 'kernel': 'poly'}
